# Pathfinding Algorithm Analysis
#### Benjamin Webster
###### November 2024

### Scenario

I have designed the "Willamette University Campus Navigation System". This system uses data from a [custom Google MyMap](https://www.google.com/maps/d/u/0/edit?mid=1O_JqDQIKP5n6X-zYpz5eD82XSd32ips&ll=44.93743410606265%2C-123.03185815866438&z=16) I created of the Willamette University campus. Pins represent building entrances/exits and paths represent walkways. The data is exported as a KML file, and updates live as changes are made to the MyMap through a web-link. This data is read in to a custom Graph datastructure defined in [graph.py](./Willamette%20University%20Campus%20Navigation%20System/graph.py), wherein the locations and paths are converted to Nodes and connected accordingly. Weights are calculated using the longitude and latitude of each node using functionality from the geopy library. Defined in [pathfinding_algorithms.py](./Willamette%20University%20Campus%20Navigation%20System/pathfinding_algorithms.py) are custom implementations of the dijkstra's, greedy best-first search, and A* pathfinding algorithms. Through the console-based UI, the user can select two locations on campus and a pathfinding algorithm. This algorithm is used to calculate the path between the locations, and the distance in feet and estimated walking time are output. The user is then given the option to visualize the path on a map, which will generate and open an HTML file displaying the path using functionality from the folium library.

Functionality has been included to load other MyMap files, including KMZ files which can be converted to KML files using the convert_to_kml function included in the [kmz_extract.py](./Willamette%20University%20Campus%20Navigation%20System/kmz_extract.py) file.

___
#### Dijkstra's Algorithm

__Parameters:__ 
- **graph**: The input graph. 
- **start**: The starting node.
- **goal**: The target node.

Dijkstra's Algorithm maintains a variable called *cost_so_far*, which represents the cumulative movement cost from the start node to each node. As the algorithm progresses, it uses *cost_so_far* to prioritize nodes in the priority queue, ensuring nodes with the lowest cost are visited first. Once the goal node is reached, the algorithm guarantees that the shortest path has been identified.

##### Optimality
Dijkstra's algorithm guarantees the shortest path to the target node, which is critical for navigation tasks. However, the algorithm may explore unnecessary paths, including suboptimal ones, especially in larger graphs. This characteristic makes Dijkstra's algorithm more suitable for scenarios where paths to multiple or all nodes need to be determined. While this inefficiency may be negligible for small graphs, it can result in significant overhead in larger graphs or repeated iterations.


##### Time Complexity: $O((E + V) \log V)$
Dijkstra's algorithm has a time complexity of $O((E + V) \log V)$, where:  
- \(V\) represents the number of vertices.  
- \(E\) represents the number of edges.  

This complexity assumes that all nodes are visited. In practice, the algorithm may terminate earlier if the goal node is reached before all nodes are explored, reducing the average-case runtime.


##### Space Complexity: $O(V)$
Dijkstra's algorithm has a space complexity of $O(V)$, where:  
- \(V\) represents the number of vertices.  

This is the worst case complexity, assuming that all nodes are visited. This linear scaling occurs because the algorithm stores the distances and priority queue entries for all vertices.


##### Implementation

In [ ]:
from graph import Graph, Node
import heapq # For the priority queue

def trace_path(start: Node, goal: Node, came_from: dict) -> list[Node]:
    """
    Traces the path from the start node to the goal node.
    """
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()

    return path

def dijkstra(graph: Graph, start: Node, goal: Node) -> tuple[int, list[Node]]:
    """
    Executes Dijkstra's algorithm and returns a tuple containing the cost of the path and the path itself.
    """
    # Trackers for the frontier, the path, and the cost so far
    # frontier uses heapq to function like a faster PriorityQueue 
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    # While there are nodes in the frontier
    while frontier:
        _, current = heapq.heappop(frontier)

        # If we've found the goal, return the cost and the path
        if current == goal:
            return (cost_so_far[goal], trace_path(start, goal, came_from))

        # Otherwise, iterate through the neighbors of the current node
        for next in graph.get_neighbors(current):
            next_node = next[0]
            new_cost = cost_so_far[current] + graph.get_weight(current, next_node)
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost # Calculates priority based only on the cost
                heapq.heappush(frontier, (priority, next_node))
                came_from[next_node] = current

    print("Goal node was not found.")
    return

___
#### Greedy Best-First Search
__Parameters:__ 
- **graph**: The input graph. 
- **start**: The starting node.
- **goal**: The target node.
- **heuritic**: The heuristic function used to determine priority.

Greedy Best First Search evaluates nodes based solely on their heuristic value, which estimates the cost to reach the goal from a given node. The algorithm prioritizes nodes with the lowest heuristic value in its priority queue. While this approach can lead to faster searches in some cases, it does not guarantee the shortest path, as it ignores the actual path cost from the start node.

##### Optimality
Greed Best First Search's heuristic tunnel vision can lead to inefficient exploration, especially when the heuristic is not well-suited to the graph structure. The algorithm can be useful in scenarios where speed is prioritized over accuracy, especially for large graphs where approximate solutions are acceptable. However, in this case, due to the nature of the scenario and the size of the graph, we can easily afford to take slightly longer to compute a path in exchange for guaranteed accuracy.

##### Time Complexity $O(b^d)$
Greedy Best-First Search has a time complexity of $O(b^d)$ where:  
- \(b\) represents the branching factor. (avereage number of neighbors)  
- \(d\) represents the depth. (length of the shortest path)

In the worst case, if the heuristic is 0 for all nodes, Greedy Best-First Search can behave like a breath-first search. With a very accurate heuristic Greedy BFS will finish much quicker, but that is unknown until runtime. Additionally, the goal node will likely be reached before all nodes arer explored, terminating the search and, reducing the average-case runtime.

##### Space Complexity: $O(V)$
Greedy Best-First Search has a space complexity of $O(V)$, where:  
- \(V\) represents the number of vertices.  

This is the worst case complexity, assuming that all nodes are visited. This linear scaling occurs because the algorithm stores the distances and priority queue entries for all vertices.


##### Implementation

In [ ]:
from graph import Graph, Node
from geopy.distance import distance, lonlat

def trace_path(start: Node, goal: Node, came_from: dict) -> list[Node]:
    """
    Traces the path from the start node to the goal node.
    """
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()

    return path

def euclidean_heuristic(current: Node, goal: Node) -> float:
    """
    Returns the Euclidean distance between the current node and the goal node.
    """
    return distance(lonlat(*current.coords), lonlat(*goal.coords)).feet

def greedy_bfs(graph: Graph, start: Node, goal: Node, heuristic = euclidean_heuristic) -> tuple[int, list[Node]]:
    """
    Executes the Greedy Best-First Search algorithm and returns a tuple containing the cost of the path and the path itself.
    """
    # Trackers for the frontier, the path, and the cost so far
    # frontier uses heapq to function like a faster PriorityQueue 
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    # While there are nodes in the frontier
    while frontier:
        _, current = heapq.heappop(frontier)

        # If we've found the goal, return the cost and the path
        if current == goal:
            return (cost_so_far[goal], trace_path(start, goal, came_from))
        
        # Otherwise, iterate through the neighbors of the current node
        for next in graph.get_neighbors(current):
            next_node = next[0]
            new_cost = cost_so_far[current] + graph.get_weight(current, next_node)
            if next_node not in came_from:
                cost_so_far[next_node] = new_cost
                priority = heuristic(next_node, goal) # Calculates priority based only on the heuristic
                heapq.heappush(frontier, (priority, next_node))
                came_from[next_node] = current
    
    print("Goal node was not found.")
    return

___
#### A*

__Parameters:__ 
- **graph**: The input graph. 
- **start**: The starting node.
- **goal**: The target node.
- **heuritic**: The heuristic function used to help determine priority.

A* combines the strengths of Dijkstra's Algorithm and Greedy Best-First Search by considering both the cumulative cost from the start node (cost_so_far) and a heuristic estimate of the remaining cost to the goal. This balance allows A* to find the shortest path efficiently, assuming the heuristic is reliable.

##### Optimality
Unlike Greedy Best-First Search, A* guarantees the shortest path to the target node, which is critical for navigation tasks. Additionally, by factoring in a heuristic estimate, A* can skip past many suboptimal paths that Dijkstra's algorithm would spend time exploring. Thus, by combining the techniques of Dijkstra's algorithm and Greedy Best-First Search, A* achieves the most optimal, quick, and consistent results.

##### Time Complexity $O(b^d)$
A* has a time complexity of $O(b^d)$ where:  
- \(b\) represents the branching factor. (avereage number of neighbors)  
- \(d\) represents the depth. (length of the shortest path)

This complexity assumes that all nodes are visited. In practice, the algorithm may terminate earlier if the goal node is reached before all nodes are explored, reducing the average-case runtime. Like Greedy BFS, A* will finish much quicker with an accurate heuristic. In a worst-case scenario, A* effectively functions exactly as quickly Dijkstra's algorithm.

##### Space Complexity: $O(V)$
A* has a space complexity of $O(V)$, where:  
- \(V\) represents the number of vertices.  

This is the worst case complexity, assuming that all nodes are visited. This linear scaling occurs because the algorithm stores the distances and priority queue entries for all vertices.

##### Implementation

In [ ]:
from graph import Graph, Node
from geopy.distance import distance, lonlat

def trace_path(start: Node, goal: Node, came_from: dict) -> list[Node]:
    """
    Traces the path from the start node to the goal node.
    """
    current = goal
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()

    return path

def euclidean_heuristic(current: Node, goal: Node) -> float:
    """
    Returns the Euclidean distance between the current node and the goal node.
    """
    return distance(lonlat(*current.coords), lonlat(*goal.coords)).feet

def a_star(graph: Graph, start: Node, goal: Node, heuristic = euclidean_heuristic) -> tuple[int, list[Node]]:
    """
    Executes the A* algorithm and returns a list of 
    nodes representing the path from start to goal.
    """
    # Trackers for the frontier, the path, and the cost so far
    # frontier uses heapq to function like a faster PriorityQueue 
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {start: None}
    cost_so_far = {start: 0}

    # While there are nodes in the frontier
    while frontier:
        _, current = heapq.heappop(frontier)

        # If we've found the goal, return the cost and the path
        if current == goal:
            return (cost_so_far[goal], trace_path(start, goal, came_from))

        # Otherwise, iterate through the neighbors of the current node
        for next in graph.get_neighbors(current):
            next_node = next[0]
            new_cost = cost_so_far[current] + graph.get_weight(current, next_node)
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic(next_node, goal) # Calculates priority based on the cost and the heuristic
                heapq.heappush(frontier, (priority, next_node))
                came_from[next_node] = current
    
    print("Goal node was not found.")
    return

#### Experimental Comparison

In order to compare the speed of the three pathfinding algorithms, I had each algorithm compute the path from each location node to each other location node (33 locations at time of writing). I then had each algorithm repeat this process 25 times, and took the average time each algorithm took to find a single path. 

**Dijkstra's Algorithm:** .0000243 seconds per iteration

**Greedy Best-First Search:** .0000733 seconds per iteration

**A\*:** 0.000462 per iteration

I was not expecting A* to perform an order of magnitude slower than Dijkstra's and Greedy Best-First Search. I hypothesize that the heuristic calculation, which relies on geospacial data to calculate the distance between two points, is likely very expensive. Because of this, Dijkstra's algorithm is the most optimal, as it takes the least amount of time and always finds the best path. In the future, I would like to experiment with larger graphs and different heuristics to see how the speed of each algorithm would be influenced.